In [1]:
import sympy as sp
import sympy.physics.mechanics as me
import sys
sys.path.insert(0, "..")
from importlib import reload
import pyMatlabFunctionQ as pyMF
reload (pyMF)

# symbols
t = sp.symbols('t')
g,c = sp.symbols('g,c')  # 
states = ['q','w']
segment = ['clavicula','scapula','humerus','ulna','radius','hand']
joints = ['quat','quat','quat','rotaxis','weld','weld']
inertia = []
mass = []
com = []
offset = []
rot_offset = []
q = []
w = []
frame = []
point_offset = []
masscenter = []
inertia_elem = []
for i,seg in enumerate(segment):
    for j in ('xx','yy','zz'):
        inertia.append(sp.symbols('I_' + seg + j))
    for j in ('x','y','z'):
        com.append(sp.symbols('com_' + seg +'_'+j))
        offset.append(sp.symbols('offset_'+seg+'_'+j))
        
    if joints[i] == 'quat':
        for j in ('x','y','z'):
            w.append(me.dynamicsymbols('w' + j+ '_' + seg))
    elif joints[i] == 'rotaxis':
        w.append(me.dynamicsymbols('w_'+ seg))
    else:
        pass
    

    if joints[i] == 'quat':
        for j in ('0','1','2','3'):
            q.append(me.dynamicsymbols('q' + j+ '_' + seg))
    elif joints[i] == 'rotaxis':
        q.append(me.dynamicsymbols('q_' + seg))
    else:
        pass

    mass.append(sp.symbols('mass_'+seg))
    frame.append(me.ReferenceFrame('frame_' + str(seg)))
    point_offset.append(me.Point('point_offset_' + str(seg)))
    masscenter.append(me.Point('masscenter_' + str(seg)))

In [2]:
# inertial frame and point
frame_ground = me.ReferenceFrame('frame_ground')
point_ground = me.Point('point_ground')
point_ground.set_vel(frame_ground,0)

offset_ground = me.Point('offset_ground')
offset_thorax = sp.symbols('offset_thoraxx:z')

offset_ground.set_pos(point_ground, offset_thorax[0]*frame_ground.x 
                      + offset_thorax[1]*frame_ground.y + offset_thorax[2]*frame_ground.z)
offset_ground.set_vel(frame_ground,0)

#rotate first body
frame[0].orient(frame_ground, 'Quaternion', q[0:4])
frame[0].set_ang_vel(frame_ground,w[0]*frame[0].x + w[1]*frame[0].y + w[2]*frame[0].z)

# set masscenter of first body
masscenter[0].set_pos(offset_ground,com[0]*frame[0].x + com[1]*frame[0].y + com[2]*frame[0].z)
masscenter[0].v2pt_theory(offset_ground,frame_ground,frame[0])

# set offset of first joint in first body
point_offset[0].set_pos(offset_ground,offset[0]*frame[0].x + offset[1]*frame[0].y + offset[2]*frame[0].z)
point_offset[0].v2pt_theory(offset_ground,frame_ground,frame[0])

# set gravity force and damping of first body
FG = [(masscenter[0], -mass[0] * g * frame_ground.y)]
DAMP = [(frame[0], -c*(w[0]*frame[0].x+w[1]*frame[0].y+w[2]*frame[0].z))]
kindeq = []
# iterate over segments 2:end (first body is already done)
for i in range(1,3):

    # orient frame w.r.t. child's frame
    frame[i].orient(frame[i-1], 'Quaternion', q[0+i*4:4+i*4])
    frame[i].set_ang_vel(frame[i-1],w[0+i*3]*frame[i].x + w[1+i*3]*frame[i].y + w[2+i*3]*frame[i].z)
        
    # set masscenter points 
    masscenter[i].set_pos(point_offset[i-1],com[0+i*3]*frame[i].x + com[1+i*3]*frame[i].y + com[2+i*3]*frame[i].z)
    masscenter[i].v2pt_theory(point_offset[i-1],frame_ground,frame[i])
    
    # set gravity force in masscenter (-y direction in frame_ground)
    FG.append((masscenter[i], -mass[i] * g * frame_ground.y))
    
    # set offsent points (where the next joint is)
    point_offset[i].set_pos(point_offset[i-1],offset[0+i*3]*frame[i].x + offset[1+i*3]*frame[i].y + offset[2+i*3]*frame[i].z)
    point_offset[i].v2pt_theory(point_offset[i-1],frame_ground,frame[i])
    
    # set damping in joints (c * angular_velocity)
    damping = -c*(w[0+i*3]*frame[i].x+w[1+i*3]*frame[i].y+w[2+i*3]*frame[i].z)
    
    # apply damping in frame, opposite moment is applied in previous frame (action and reaction)
    DAMP.append((frame[i], damping))
    DAMP.append((frame[i-1], -damping))
    # set kinematic differential equations (q_dot = f(q,u))
    
for i in range(0,3):
    kindeq.append(q[0+i*4].diff(t) - 0.5 * (-w[0+i*3]*q[1+i*4] - w[1+i*3]*q[2+i*4] - w[2+i*3]*q[3+i*4]))
    kindeq.append(q[1+i*4].diff(t) - 0.5 * (w[0+i*3]*q[0+i*4] + w[2+i*3]*q[2+i*4] - w[1+i*3]*q[3+i*4]))
    kindeq.append(q[2+i*4].diff(t) - 0.5 * (w[1+i*3]*q[0+i*4] - w[2+i*3]*q[1+i*4] + w[0+i*3]*q[3+i*4]))
    kindeq.append(q[3+i*4].diff(t) - 0.5 * (w[2+i*3]*q[0+i*4] + w[1+i*3]*q[1+i*4] - w[0+i*3]*q[2+i*4]))

In [3]:
# symbols for ulna
offset_humerus_rot = sp.symbols('offset_humerus_rotx:z')
ulna_rot_frame = me.ReferenceFrame('ulna_rot_frame')
EL_rot_axis = sp.symbols('EL_rot_axisx:z')
PSY_rot_axis = sp.symbols('PSY_rot_axisx:z')

# offset frame rotated in humerus frame (frame[2])
ulna_rot_frame.orient_axis(frame[2],frame[2].z,offset_humerus_rot[2])

In [4]:
# ulna and elbow joint
frame[3].orient_axis(ulna_rot_frame,ulna_rot_frame.x*EL_rot_axis[0]
                     +ulna_rot_frame.y*EL_rot_axis[1]+ulna_rot_frame.z*EL_rot_axis[2],
                     q[12])

frame[3].set_ang_vel(ulna_rot_frame,
                      w[9]*(ulna_rot_frame.x*EL_rot_axis[0]+ulna_rot_frame.y*EL_rot_axis[1]
                            +ulna_rot_frame.z*EL_rot_axis[2]))

masscenter[3].set_pos(point_offset[2],com[0+3*3]*frame[3].x + com[1+3*3]*frame[3].y + com[2+3*3]*frame[3].z)
masscenter[3].v2pt_theory(point_offset[2],frame_ground,frame[3])
FG.append((masscenter[3], -mass[3] * g * frame_ground.y))

In [5]:
DAMP.append(((frame[3]),-c*w[9]*(ulna_rot_frame.x*EL_rot_axis[0]+ulna_rot_frame.y*EL_rot_axis[1]
                        +ulna_rot_frame.z*EL_rot_axis[2])))
DAMP.append((ulna_rot_frame,c*w[9]*(ulna_rot_frame.x*EL_rot_axis[0]+ulna_rot_frame.y*EL_rot_axis[1]
                            +ulna_rot_frame.z*EL_rot_axis[2])))
kindeq.append(w[9]-q[12].diff(t))

In [6]:
point_offset[3].set_pos(point_offset[2],offset[0+3*3]*frame[3].x + offset[1+3*3]*frame[3].y + offset[2+3*3]*frame[3].z)
point_offset[3].v2pt_theory(point_offset[2],frame_ground,frame[3]);

In [7]:
# radius and PSY joint (weld joint for now)
frame[4].orient_axis(frame[3],frame[3].z,0)
frame[4].set_ang_vel(frame[3],0)
masscenter[4].set_pos(point_offset[3],com[0+4*3]*frame[4].x + com[1+4*3]*frame[4].y + com[2+4*3]*frame[4].z)
masscenter[4].v2pt_theory(point_offset[3],frame_ground,frame[4])
FG.append((masscenter[4], -mass[4] * g * frame_ground.y))
frame[4].ang_vel_in(frame[3])

0

In [8]:
point_offset[4].set_pos(point_offset[3],offset[0+4*3]*frame[3].x + offset[1+4*3]*frame[3].y + offset[2+4*3]*frame[3].z)
point_offset[4].v2pt_theory(point_offset[3],frame_ground,frame[3])

# hand
frame[5].orient_axis(frame[4],frame[4].z,0)
frame[5].set_ang_vel(frame[4],0)
masscenter[5].set_pos(point_offset[4],com[0+5*3]*frame[5].x + com[1+5*3]*frame[5].y + com[2+5*3]*frame[5].z)
masscenter[5].v2pt_theory(point_offset[4],frame_ground,frame[5])
FG.append((masscenter[5], -mass[5] * g * frame_ground.y))

BODY = []

for i in range(len(segment)):
    # set inertias of each body and create RigidBodies
    I = me.inertia(frame[i], inertia[0+i*3], inertia[1+i*3], inertia[2+i*3])
    BODY.append(me.RigidBody('body' + str(i), masscenter[i], frame[i], mass[i], (I, masscenter[i])))

In [9]:
# Contact between scapula and thorax
contTS = sp.symbols('contTS1:4')
contAI = sp.symbols('contAI1:4')
elips_trans = sp.symbols('elips_trans1:4')
elips_dim = sp.symbols('elips_dim1:4')
k_contact_in, eps_in = sp.symbols('k_contact_in eps_in')
k_contact_out, eps_out = sp.symbols('k_contact_out eps_out')
second_elips_dim = sp.Symbol('second_elips_dim')

# contact points 
contact_point1 = me.Point('CP1')
contact_point1.set_pos(point_offset[0],contTS[0]*frame[1].x+contTS[1]*frame[1].y  +contTS[2]*frame[1].z)
# contact_point1.set_vel(scapula.frame,0) # point is fixed in scapula
contact_point1.v2pt_theory(point_offset[0],frame_ground,frame[1])

contact_point2 = me.Point('CP2')
contact_point2.set_pos(point_offset[0],contAI[0]*frame[1].x+contAI[1]*frame[1].y  +contAI[2]*frame[1].z)
# contact_point2.set_vel(scapula.frame,0) # point is fixed in scapula
contact_point2.v2pt_theory(point_offset[0],frame_ground,frame[1])

## contact forces

# Distances between contact points and thorax frame
x_pos1 = contact_point1.pos_from(point_ground).dot(frame_ground.x)
y_pos1 = contact_point1.pos_from(point_ground).dot(frame_ground.y)
z_pos1 = contact_point1.pos_from(point_ground).dot(frame_ground.z)
x_pos2 = contact_point2.pos_from(point_ground).dot(frame_ground.x)
y_pos2 = contact_point2.pos_from(point_ground).dot(frame_ground.y)
z_pos2 = contact_point2.pos_from(point_ground).dot(frame_ground.z)

# Contact forces
f1_in = ((x_pos1-elips_trans[0])/elips_dim[0])**2+((y_pos1-elips_trans[1])/elips_dim[1])**2+((z_pos1-elips_trans[2])/elips_dim[2])**2-1
f1_out = ((x_pos1-elips_trans[0])/(second_elips_dim*elips_dim[0]))**2+((y_pos1-elips_trans[1])/(second_elips_dim*elips_dim[1]))**2+((z_pos1-elips_trans[2])/(second_elips_dim*elips_dim[2]))**2-1
F1_in = 1/2*(f1_in-sp.sqrt(f1_in**2+eps_in**2))
F1_out = 1/2*(f1_out+sp.sqrt(f1_out**2+eps_out**2))
Fx1 = -(k_contact_in*F1_in+k_contact_out*F1_out)*(x_pos1-elips_trans[0])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[0]**2)
Fy1 = -(k_contact_in*F1_in+k_contact_out*F1_out)*(y_pos1-elips_trans[1])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[1]**2)
Fz1 = -(k_contact_in*F1_in+k_contact_out*F1_out)*(z_pos1-elips_trans[2])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[2]**2)

f2_in = ((x_pos2-elips_trans[0])/elips_dim[0])**2+((y_pos2-elips_trans[1])/elips_dim[1])**2+((z_pos2-elips_trans[2])/elips_dim[2])**2-1
f2_out = ((x_pos2-elips_trans[0])/(second_elips_dim*elips_dim[0]))**2+((y_pos2-elips_trans[1])/(second_elips_dim*elips_dim[1]))**2+((z_pos2-elips_trans[2])/(second_elips_dim*elips_dim[2]))**2-1
F2_in = 1/2*(f2_in-sp.sqrt(f2_in**2+eps_in**2))
F2_out = 1/2*(f2_out+sp.sqrt(f2_out**2+eps_out**2))
Fx2 = -(k_contact_in*F2_in+k_contact_out*F2_out)*(x_pos2-elips_trans[0])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[0]**2)
Fy2 = -(k_contact_in*F2_in+k_contact_out*F2_out)*(y_pos2-elips_trans[1])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[1]**2)
Fz2 = -(k_contact_in*F2_in+k_contact_out*F2_out)*(z_pos2-elips_trans[2])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[2]**2)

# applying contact forces to contact points in thorax frame
# scapula.apply_force(thorax.x*Fx1+thorax.y*Fy1+thorax.z*Fz1,contact_point1)
# scapula.apply_force(thorax.x*Fx2+thorax.y*Fy2+thorax.z*Fz2,contact_point2)

cont_force1 = [(contact_point1,frame_ground.x*Fx1+frame_ground.y*Fy1+frame_ground.z*Fz1)]
cont_force2 = [(contact_point2,frame_ground.x*Fx2+frame_ground.y*Fy2+frame_ground.z*Fz2)]
CONT = cont_force1+cont_force2


In [10]:
KM = me.KanesMethod(frame_ground, q_ind=q, u_ind=w, kd_eqs=kindeq)
(fr, frstar) = KM.kanes_equations(BODY, (FG+DAMP+CONT))

In [11]:
import pyMatlabFunctionQ as pyMF
reload (pyMF)
body_constants = {'I_': inertia,'mass_':mass,'com_':com,'offset_':offset,'c': c,'g': g}
other_constants = {'offset_humerus_rot':list(offset_humerus_rot),'EL_rot_axis': list(EL_rot_axis),
                    'k_contact_in': k_contact_in,'eps_in': eps_in,'contTS': list(contTS),
                    'contAI': list(contAI), 'elips_trans':list(elips_trans), 'elips_dim':list(elips_dim),
                    'k_contact_out': k_contact_out,'eps_out': eps_out, 'second_elips_dim': second_elips_dim,
                    'offset_thorax': list(offset_thorax)}
usubs = sp.symbols('u1:11')
qsubs = sp.symbols('q0:13')

states = [qsubs,usubs]
subs_q = {q[i]: qsubs[i] for i in range(len(q))}
subs_u = {w[i]: usubs[i] for i in range(len(w))}
mm = me.msubs(KM.mass_matrix_full,subs_q,subs_u)
fo = me.msubs(KM.forcing_full,subs_q,subs_u)

pyMF.MatlabFunction(function = mm,
                    fun_name = 'mm_Q',assignto = 'mm',
                    states = states,body_constants = body_constants,segments = segment,other_constants=other_constants)
pyMF.MatlabFunction(function = fo,
                    fun_name = 'fo_Q',assignto = 'fo',
                    states = states,body_constants = body_constants,segments = segment,other_constants=other_constants)